In [155]:
import os
from dotenv import load_dotenv

load_dotenv()
GROK_API_KEY= os.getenv("GROK_API_KEY")
NEO4J_URI= os.getenv("NEO4J_URI")
NEO4J_USER= os.getenv("NEO4J_USER")
NEO4J_PASSWORD= os.getenv("NEO4J_PASSWORD")


In [156]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USER, password=NEO4J_PASSWORD,refresh_schema=False)

# Run a simple query to test the connection
graph

In [157]:
movie_Query = """
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row

MERGE (m:Movie {id: row.movieId})
SET m.title = row.title,
    m.released = date(row.released),
    m.imdbRating = toFloat(row.imdbRating)

FOREACH (director IN split(row.director, '|') |
  MERGE (p:Person {name: trim(director)})
  MERGE (p)-[:DIRECTED]->(m)
)

FOREACH (actor IN split(row.actors, '|') |
  MERGE (p:Person {name: trim(actor)})
  MERGE (p)-[:ACTED_IN]->(m)
)

FOREACH (genre IN split(row.genres, '|') |
  MERGE (g:Genre {name: trim(genre)})
  MERGE (m)-[:IN_GENRE]->(g)
)
"""


In [158]:
def safe_query(self, query, params=None):
    with self._driver.session(database=self._database) as session:
        result = session.run(query, params or {})
        return [r.data() for r in result]
    
Neo4jGraph.query = safe_query

In [159]:
graph.query(movie_Query)

[]

In [160]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, title: STRING, released: DATE, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [161]:
from langchain_groq import ChatGroq

model=ChatGroq(model="gemma2-9b-it", api_key=GROK_API_KEY)

In [199]:
from langchain_core.prompts import PromptTemplate

# --- Schema string ---
schema = (
    "Labels: Person, Movie, Genre\n"
    "Relationships:\n"
    "  (Person)-[:ACTED_IN]->(Movie)\n"
    "  (Person)-[:DIRECTED]->(Movie)\n"
    "  (Movie)-[:IN_GENRE]->(Genre)\n"
    "Properties: Movie.title (string), Movie.imdbRating (float), "
    "Movie.released (date), Person.name (string)\n"
)

# --- Prompt template ---
prompt = PromptTemplate(
    input_variables=["question", "schema"],
    template=(
        "Schema:\n{schema}\n\n"
        "Write a Cypher query for the question below.\n\n"
        "Rules:\n"
        "- Use labels exactly: Person, Movie, Genre.\n"
        "- Use relationships exactly with correct directions:\n"
        "    (Person)-[:ACTED_IN]->(Movie)\n"
        "    (Person)-[:DIRECTED]->(Movie)\n"
        "    (Movie)-[:IN_GENRE]->(Genre)\n"
        "- Put relationship patterns in MATCH, not in WHERE.\n"
        "- Use WHERE only for scalar comparisons.\n"
        "- Always alias returned values.\n"
        "- Alias aggregates (e.g. count(m) AS cnt).\n"
        "- Do not invent labels, properties, or relationships.\n\n"
        "Q: {question}\nCypher:"
    )
)


In [200]:
prompt

PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Schema:\n{schema}\n\nWrite a Cypher query for the question below.\n\nRules:\n- Use labels exactly: Person, Movie, Genre.\n- Use relationships exactly with correct directions:\n    (Person)-[:ACTED_IN]->(Movie)\n    (Person)-[:DIRECTED]->(Movie)\n    (Movie)-[:IN_GENRE]->(Genre)\n- Put relationship patterns in MATCH, not in WHERE.\n- Use WHERE only for scalar comparisons.\n- Always alias returned values.\n- Alias aggregates (e.g. count(m) AS cnt).\n- Do not invent labels, properties, or relationships.\n\nQ: {question}\nCypher:')

In [201]:
print(prompt.format(question="How many artists are there?", schema=schema))

Schema:
Labels: Person, Movie, Genre
Relationships:
  (Person)-[:ACTED_IN]->(Movie)
  (Person)-[:DIRECTED]->(Movie)
  (Movie)-[:IN_GENRE]->(Genre)
Properties: Movie.title (string), Movie.imdbRating (float), Movie.released (date), Person.name (string)


Write a Cypher query for the question below.

Rules:
- Use labels exactly: Person, Movie, Genre.
- Use relationships exactly with correct directions:
    (Person)-[:ACTED_IN]->(Movie)
    (Person)-[:DIRECTED]->(Movie)
    (Movie)-[:IN_GENRE]->(Genre)
- Put relationship patterns in MATCH, not in WHERE.
- Use WHERE only for scalar comparisons.
- Always alias returned values.
- Alias aggregates (e.g. count(m) AS cnt).
- Do not invent labels, properties, or relationships.

Q: How many artists are there?
Cypher:


In [202]:

from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm( graph=graph, llm=model,verbose=True,allow_dangerous_requests=True,cypher_prompt=prompt)

chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x000001A3842B3620>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Schema:\n{schema}\n\nWrite a Cypher query for the question below.\n\nRules:\n- Use labels exactly: Person, Movie, Genre.\n- Use relationships exactly with correct directions:\n    (Person)-[:ACTED_IN]->(Movie)\n    (Person)-[:DIRECTED]->(Movie)\n    (Movie)-[:IN_GENRE]->(Genre)\n- Put relationship patterns in MATCH, not in WHERE.\n- Use WHERE only for scalar comparisons.\n- Always alias returned values.\n- Alias aggregates (e.g. count(m) AS cnt).\n- Do not invent labels, properties, or relationships.\n\nQ: {question}\nCypher:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A38496DE80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A3844A302

In [212]:
chain.run({"query": "How many artists are there?", "schema": schema})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)
RETURN count(p) AS artistCount 

Full Context:
[{'artistCount': 1239}]

> Finished chain.


'There are 1239 artists. \n'

In [213]:
# --- Test queries ---
q = [
    "How many movies are there?",
    "List all actors who played in 'Casino'.",
    "Which movies did Tom Hanks act in?",
    "Who directed 'Toy Story'?",
    "List all genres of the movie 'Jumanji'."
]



# --- Run queries ---
for query in q:

    result = chain.run({"query": query, "schema": schema})
    print("Q: " + query)
    print(result)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)
RETURN count(m) AS movieCount

Full Context:
[{'movieCount': 299}]

> Finished chain.
Q: How many movies are there?
There are 299 movies. 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: 'Casino'}) 
RETURN DISTINCT p.name AS actor

Full Context:
[{'actor': 'Robert De Niro'}, {'actor': 'Joe Pesci'}, {'actor': 'Sharon Stone'}, {'actor': 'James Woods'}]

> Finished chain.
Q: List all actors who played in 'Casino'.
Robert De Niro, Joe Pesci, Sharon Stone, and James Woods. 



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m:Movie)
RETURN m.title AS movie_title 

Full Context:
[{'movie_title': 'Toy Story'}, {'movie_title': 'Apollo 13'}]

> Finished chain.
Q: Which movies did Tom Hanks act in?
Toy Story, Apollo 13 



> Entering new GraphCypherQAChain chain...
Generate

In [215]:
result = chain.run({"query": "List all the movies which have a rating greater than 8 and genre Comedy.", "schema": schema})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre {name: 'Comedy'})
WHERE m.imdbRating > 8
RETURN m.title AS movie_title

Full Context:
[{'movie_title': 'Toy Story'}, {'movie_title': 'Pulp Fiction'}]

> Finished chain.


In [216]:
result = chain.run({"query": "List all the movies which have the genre Comedy.", "schema": schema})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre {name: 'Comedy'})
RETURN m.title AS movieTitle

Full Context:
[{'movieTitle': 'Toy Story'}, {'movieTitle': 'Grumpier Old Men'}, {'movieTitle': 'Waiting to Exhale'}, {'movieTitle': 'Father of the Bride Part II'}, {'movieTitle': 'Sabrina'}, {'movieTitle': 'American President, The'}, {'movieTitle': 'Dracula: Dead and Loving It'}, {'movieTitle': 'Four Rooms'}, {'movieTitle': 'Ace Ventura: When Nature Calls'}, {'movieTitle': 'Money Train'}]

> Finished chain.
